In [1]:
import pandas as pd
import altair as alt

In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/Krow10/code4rena-scraper/master/github_code4rena.csv")
data["date"] = pd.to_datetime(data["date"])

### Risk count
Count for each wardens the number of submissions sent by severity

In [4]:
plt_data = data.groupby("handle")["risk"].value_counts().to_frame("risk_count").reset_index()
plt_data = pd.merge(
    plt_data, 
    plt_data.groupby("handle").sum().rename(columns={'risk_count':'total_entries'})["total_entries"], 
    how="left", 
    on="handle"
)
plt_data["risk"] = plt_data["risk"].map({'3': 'High', '2': 'Med', '1': 'Low', '0': 'QA', 'G': 'Gas optimization', 'Q': 'QA'})

/tmp/ipykernel_2019/1153919903.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  plt_data.groupby("handle").sum().rename(columns={'risk_count':'total_entries'})["total_entries"],


### Total entries selector
Allow to filter wardens by the number of entries they submitted

In [5]:
slider = alt.binding_range(
    min=0, 
    max=plt_data["total_entries"].max(),
    step=1, 
    name='Minimum number of warden submissions:'
)
total_entries_selector = alt.selection_single(
    bind=slider,
    name="total_entries_selector", 
    fields=['cutoff'], 
    init={'cutoff': 0}
)

### Multi risk selector
Allows to filter submissions by risk severity 

In [6]:
multi_risk_selector = alt.selection_multi(fields=['risk'])

In [7]:
ordered_legend_risk_labels = ['High', 'Med', 'Low', 'Gas optimization', 'QA']
colors = ["#f94144","#f8961e","#f9c74f","#90be6d","#43aa8b","#577590"]

In [8]:
base = alt.Chart(plt_data).transform_calculate(
    order="{'QA': 0, 'Gas optimization': 1, 'Low': 2, 'Med': 3, 'High': 4}[datum.risk]"
)

This chart will show the number of submissions for each wardens, grouped by severity

In [9]:
warden_bars = base.mark_bar(size=20).encode(
    x=alt.X(
        'handle:N', 
        axis=alt.Axis(title='Wardens', grid=False, labelAngle=-45), 
        sort=alt.EncodingSortField(field="total_entries", order="descending")
    ),
    y=alt.Y(
        'risk_count:Q', 
        stack='zero',
        axis=alt.Axis(title='Total entries count')
    ),
    color=alt.Color(
        'risk:N', 
        title='Risk level',
        scale=alt.Scale(domain=ordered_legend_risk_labels, range=colors),
        legend=alt.Legend(title='Risk severity level', values=ordered_legend_risk_labels)
    ),
    order='order:O',
    tooltip=['handle', 'risk:N', 'risk_count']
).add_selection(
    total_entries_selector
).transform_filter(
    alt.datum.total_entries > total_entries_selector.cutoff
).transform_filter(
    multi_risk_selector
).properties(width=alt.Step(25))

This chart show the total number of submissions for each severity category.
It allows filtering the data by selecting one (left click) or multiple (shift + left click) severity categories.

In [10]:
risk_bars = base.mark_bar(size=30).encode(
    x=alt.X(
        'risk_count',
        aggregate='sum',
        axis=alt.Axis(title='Total entries')
    ),
    y=alt.Y(
        'risk:O',
        axis=alt.Axis(title='Risk severity level'),
        sort=ordered_legend_risk_labels
    ),
    color=alt.condition(
        multi_risk_selector, 
        alt.Color('risk:O', title='Risk level', legend=None), 
        alt.value('lightgray')
    ),
    tooltip=['sum(risk_count)']
).add_selection(
    multi_risk_selector,
    total_entries_selector
).transform_filter(
    alt.datum.total_entries > total_entries_selector.cutoff
).properties(height=alt.Step(35))

In [11]:
(warden_bars & risk_bars).configure_axis(
    titleFontSize=14,
    labelFontSize=12
)

/opt/hostedtoolcache/Python/3.11.1/x64/lib/python3.11/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)